In [1]:
import matplotlib.pyplot as plt
import uproot
import numpy as np
import pandas as pd

from pathlib import Path
from sklearn.model_selection import train_test_split

from datetime import datetime
import sys
sys.path.insert(1, '/afs/desy.de/user/a/axelheim/private/MC_studies/Dstlnu_Bt_generic/util_funcs/')
from pandas_colFuncs import isBtoDstlnu, whichBisSig, customMCmatching, B_ID

In [2]:
now = datetime.now()
print("time at start =", now)

time at start = 2021-10-01 09:01:32.963320


In [3]:
HTCondorRun = str(sys.argv[1])
print("HTCondorRun:",HTCondorRun)

HTCondorRun: -f


In [14]:
sub = str(sys.argv[2])
if sub.find("sub") == -1:
    sub = "sub00"
print("used sub:", sub)

used sub: sub00


In [30]:
save_data = False
save_preprocessedDataframe = True
tmp_data = True
take_subset = False
subset_size = 100000

In [15]:
nfs_path = "/nfs/dust/belle2/user/axelheim/MC_studies/Dstlnu_Bt_generic/"

data_subdir = "Dstlnu_SHR_BsX/"
root_subdir = "SHR_dataSteering_run1/"   

root_path = nfs_path + "SHR_Hc_correctReco_BsX/" + root_subdir + sub + "/"

In [6]:
merged = "merged_"
if tmp_data:
    merged += "tmp_"

In [7]:
fileY4S = uproot.open(root_path + merged + "DXtagDstl.root:variables")
#afsPath = "/afs/desy.de/user/a/axelheim/private/MC_studies/Dstlnu_Bt_generic/SHR_Hc_correctReco_BsX/SHR_dataSteering_run1/"
#fileY4S = uproot.open(afsPath + "DXtagDstl.root:variables")

In [8]:
names = ["gammas","electrons","pions","kaons","muons"]
dfs = []
for name in names:
    filename = root_path + merged + "{}.root:variables".format(name)
    #filename = afsPath + "{}.root:variables".format(name)
    print(filename)
    tmpFileFSPs = uproot.open(filename)
    df_tmp = tmpFileFSPs.arrays(library="pd")
    dfs.append(df_tmp)

/nfs/dust/belle2/user/axelheim/MC_studies/Dstlnu_Bt_generic/SHR_Hc_correctReco_BsX/SHR_dataSteering_run1/merged_tmp_gammas.root:variables
/nfs/dust/belle2/user/axelheim/MC_studies/Dstlnu_Bt_generic/SHR_Hc_correctReco_BsX/SHR_dataSteering_run1/merged_tmp_electrons.root:variables
/nfs/dust/belle2/user/axelheim/MC_studies/Dstlnu_Bt_generic/SHR_Hc_correctReco_BsX/SHR_dataSteering_run1/merged_tmp_pions.root:variables
/nfs/dust/belle2/user/axelheim/MC_studies/Dstlnu_Bt_generic/SHR_Hc_correctReco_BsX/SHR_dataSteering_run1/merged_tmp_kaons.root:variables
/nfs/dust/belle2/user/axelheim/MC_studies/Dstlnu_Bt_generic/SHR_Hc_correctReco_BsX/SHR_dataSteering_run1/merged_tmp_muons.root:variables


In [9]:
df_FSPs = pd.concat(dfs)

In [10]:
df_Y4S = fileY4S.arrays(library="pd")

In [11]:
print(df_FSPs.shape[0])
print(df_Y4S.shape[0])

18876768
400786


In [12]:
# delete FSPs for which no Y4S file entry was found
df_FSPs = df_FSPs[df_FSPs['__event__'].isin(df_Y4S["__event__"])]

In [13]:
print(df_FSPs.shape[0])

17068618


### delete particles which occur more than ones based on uniqueParticleIdentifier

In [16]:
groupsFSPs_uniqParID = pd.DataFrame({'count' : df_FSPs.groupby( ["__event__","uniqueParticleIdentifier"] ).size()}).reset_index()
#with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
#    print(groupsFSPs_uniqParID.sort_values("count"))

In [17]:
print("df_FSPs.shape[0]:",df_FSPs.shape[0])
print("groupsFSPs_uniqParID.shape[0]:",groupsFSPs_uniqParID.shape[0])
print("df_Y4S.shape[0]:",df_Y4S.shape[0])

df_FSPs.shape[0]: 17068618
groupsFSPs_uniqParID.shape[0]: 12716491
df_Y4S.shape[0]: 400786


In [18]:
# delete particles which occur more than ones (keep first) and if possible keep the one with basf2_used==1
print("df_FSPs[basf2_used].value_counts():",df_FSPs["basf2_used"].value_counts())
df_FSPs = df_FSPs.sort_values("basf2_used",ascending=False).drop_duplicates(subset=("__event__","uniqueParticleIdentifier"), keep='first')
print("df_FSPs[basf2_used].value_counts():",df_FSPs["basf2_used"].value_counts())

df_FSPs[basf2_used].value_counts(): 0.0    9087454
1.0    7981164
Name: basf2_used, dtype: int64
df_FSPs[basf2_used].value_counts(): 1.0    7971590
0.0    4744901
Name: basf2_used, dtype: int64


In [19]:
print("df_FSPs.shape[0]:",df_FSPs.shape[0])

df_FSPs.shape[0]: 12716491


## take a sample if used in notebook for faster processing

In [14]:
all_evt_nums = df_FSPs['__event__'].unique()
all_evt_nums.shape[0]

90

In [15]:
sample_evt_nums = np.random.choice(all_evt_nums, size=subset_size)
sample_evt_nums.shape[0]

100000

In [16]:
if take_subset:
    df_FSPssample = df_FSPs[df_FSPs['__event__'].isin(sample_evt_nums)]
    #df_Y4S=df_Y4Ssample
    df_FSPs=df_FSPssample

In [17]:
print("df_FSPs.shape[0]:",df_FSPs.shape[0])
print("numEvents:",df_FSPs['__event__'].unique().shape[0])

df_FSPs.shape[0]: 4215
numEvents: 90


## save sample 

In [22]:
if take_subset:
    df_FSPs.to_csv(root_path + "df_FSPs_sample_{}_evts.csv".format(subset_size))
    #df_Y4S.to_csv(root_path + "df_Y4S_sample__evts.csv")

In [23]:
load = False
if load:
    df_FSPs = pd.read_csv(root_path + "df_FSPs_sample_{}_evts.csv".format(subset_size))
    #df_Y4S = pd.read_csv(root_path + "df_Y4S_sample10evts.csv")

## filter the wanted D*lnu and Hc isSignal==1 events

In [20]:
df_Y4S['isBtoDstlnu'] = df_Y4S.apply(isBtoDstlnu, axis=1)

In [21]:
df_Y4S[(df_Y4S['isBtoDstlnu'] == 1) & (df_Y4S['Hc_isSignalAcceptMissingGamma'] == 1.0)].shape[0]

14945

In [22]:
df_Y4S = df_Y4S[(df_Y4S['isBtoDstlnu'] == 1) & (df_Y4S['Hc_isSignalAcceptMissingGamma'] == 1.0)]

In [26]:
# delete FSPs for which no Y4S file entry is left after filer
df_FSPs = df_FSPs[df_FSPs['__event__'].isin(df_Y4S["__event__"])]
df_FSPs.shape[0]

416921

### add cols with extra info for data prod

In [24]:
df_Y4S['Bsig_uniqParID'] = df_Y4S.apply(whichBisSig, axis=1)

In [25]:
# function to create col with the particles mother B's uniqueParticleIdentifier
df_FSPs['B_ID'] = df_FSPs.apply(B_ID, axis=1)

### print one event

In [25]:
df_Y4S[(df_Y4S['isBtoDstlnu'] == 1) & (df_Y4S['Hc_isSignalAcceptMissingGamma'] == 1.0)][["__event__",'Bsig_uniqParID','isBtoDstlnu','Hc_isSignalAcceptMissingGamma','Hc_mcPDG','Hc_genMotherPDG','Hc_uniqParID','Hc_genMotherID']]

,__event__,Bsig_uniqParID,isBtoDstlnu,Hc_isSignalAcceptMissingGamma,Hc_mcPDG,Hc_genMotherPDG,Hc_uniqParID,Hc_genMotherID
62,2338198,83886081.0,1,1.0,411.0,-511.0,100663296.0,2.0
66,2338716,83886082.0,1,1.0,-411.0,511.0,100663296.0,1.0


In [33]:
df_FSPs[df_FSPs["__event__"] == 2338198].sort_values('mcPDG').sort_values('B_ID',ascending=False)[['mcPDG',
'B_ID','genMothPDG_0', 'mcMother0_uniqParID', 'genMotherID_0',
'genMothPDG_1', 'mcMother1_uniqParID', 'genMotherID_1',
'genMothPDG_2', 'mcMother2_uniqParID', 'genMotherID_2',
'genMothPDG_3', 'mcMother3_uniqParID', 'genMotherID_3',
'genMothPDG_4', 'mcMother4_uniqParID', 'genMotherID_4',
'genMothPDG_5', 'mcMother5_uniqParID', 'genMotherID_5',
'genMothPDG_6', 'mcMother6_uniqParID', 'genMotherID_6',
'genMothPDG_7', 'mcMother7_uniqParID', 'genMotherID_7',
'genMothPDG_8', 'mcMother8_uniqParID', 'genMotherID_8',
'genMothPDG_9', 'mcMother9_uniqParID', 'genMotherID_9']]

,mcPDG,B_ID,genMothPDG_0,mcMother0_uniqParID,genMotherID_0,genMothPDG_1,mcMother1_uniqParID,genMotherID_1,genMothPDG_2,mcMother2_uniqParID,...,genMotherID_6,genMothPDG_7,mcMother7_uniqParID,genMotherID_7,genMothPDG_8,mcMother8_uniqParID,genMotherID_8,genMothPDG_9,mcMother9_uniqParID,genMotherID_9
765,-211.0,83886082,310.0,83886101.0,21.0,-311.0,83886090.0,10.0,-511.0,83886082.0,...,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN,0.0
1426,22.0,83886082,111.0,83886093.0,13.0,-511.0,83886082.0,2.0,300553.0,83886080.0,...,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN,0.0
1417,22.0,83886082,111.0,83886098.0,18.0,411.0,83886087.0,7.0,-511.0,83886082.0,...,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN,0.0
1421,22.0,83886082,111.0,83886098.0,18.0,411.0,83886087.0,7.0,-511.0,83886082.0,...,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN,0.0
1423,22.0,83886082,111.0,83886093.0,13.0,-511.0,83886082.0,2.0,300553.0,83886080.0,...,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN,0.0
1424,22.0,83886082,111.0,83886088.0,8.0,-511.0,83886082.0,2.0,300553.0,83886080.0,...,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN,0.0
756,211.0,83886082,310.0,83886101.0,21.0,-311.0,83886090.0,10.0,-511.0,83886082.0,...,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN,0.0
1428,22.0,83886082,111.0,83886088.0,8.0,-511.0,83886082.0,2.0,300553.0,83886080.0,...,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN,0.0
752,211.0,83886082,411.0,83886087.0,7.0,-511.0,83886082.0,2.0,300553.0,83886080.0,...,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN,0.0
1432,211.0,83886082,321.0,83886091.0,11.0,-511.0,83886082.0,2.0,300553.0,83886080.0,...,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN,0.0


## check if category combinations make sense

In [27]:
groupsAllFSPs = pd.DataFrame({'count' : df_FSPs.groupby( ["basf2_used","basf2_Bsig","basf2_X"] ).size()}).reset_index()
groupsAllFSPs

,basf2_used,basf2_Bsig,basf2_X,count
0,0.0,0.0,0.0,154089
1,0.0,1.0,0.0,54
2,1.0,0.0,0.0,48518
3,1.0,0.0,1.0,123958
4,1.0,1.0,0.0,90302


## data Saving

### save dataframes on NFS

In [32]:
if HTCondorRun == "isHTCondorRun" or save_preprocessedDataframe == True:
    df_FSPs.to_csv(root_path + "df_FSPs_preProcessed_SHR.csv")
    df_Y4S.to_csv(root_path + "df_Y4S_preProcessed_SHR.csv")

### load dataframes from different subs and concat them

In [37]:
df_FSPs_final = df_FSPs
df_Y4S_final = df_Y4S

In [39]:
print(df_FSPs_final.shape[0])
print(df_Y4S_final.shape[0])

416921
14945


In [35]:
df_FSPs_list=[]
df_Y4S_list=[]
for sub in ["sub00","sub01","sub02"]:
    root_path = nfs_path + "SHR_Hc_correctReco_BsX/" + root_subdir + sub + "/"    
    df_FSPs = pd.read_csv(root_path + "df_FSPs_preProcessed_SHR.csv")
    df_FSPs_list.append(df_FSPs)
    df_Y4S = pd.read_csv(root_path + "df_Y4S_preProcessed_SHR.csv")
    df_Y4S_list.append(df_Y4S)
    
df_FSPs_final = pd.concat(df_FSPs_list)
df_Y4S_final = pd.concat(df_Y4S_list)

FileNotFoundError: [Errno 2] No such file or directory: '/nfs/dust/belle2/user/axelheim/MC_studies/Dstlnu_Bt_generic/SHR_Hc_correctReco_BsX/SHR_dataSteering_run1/sub01/df_FSPs_preProcessed_SHR.csv'

# start of NN data creation

In [40]:
numFSPs = pd.DataFrame({'count' : df_FSPs_final.groupby( ["__event__"] ).size()}).reset_index()

minFSPs = numFSPs["count"].min()
maxFSPs = numFSPs["count"].max()
print("minFSPs:",minFSPs)
print("maxFSPs:",maxFSPs,'\n')

df_FSPs_final['numFSPs'] = df_FSPs_final.groupby('__event__')['__event__'].transform('count')

minFSPs: 1
maxFSPs: 70 



In [41]:
data_dir = Path(nfs_path + "data/" + data_subdir + root_subdir)    
if save_data:
    data_dir.mkdir(parents=True, exist_ok=True)
print("Will save data to:", data_dir,'is', save_data ,'\n')

Will save data to: /nfs/dust/belle2/user/axelheim/MC_studies/Dstlnu_Bt_generic/data/Dstlnu_SHR_BsX_100000_evts/SHR_dataSteering_run1 is False 



In [ ]:
for num_FSPs_toData in range(minFSPs, maxFSPs+1):
    df_num_subset = df_FSPs_final.copy()
    df_num_subset = df_num_subset[df_num_subset['numFSPs'] == num_FSPs_toData]
    
        
    numEvents = df_num_subset.__event__.nunique()
    print("numEvents:",numEvents)
    print("num_FSPs_toData:",num_FSPs_toData)  
    if numEvents == 0:
        print("skipped because empty \n")
        continue

    if numEvents < 10:
        print("skipped because <10 events \n")
        continue
    
    num_features = 4
    leaves = np.zeros((numEvents, num_FSPs_toData,  num_features))  
    SA_target =  np.zeros((numEvents, num_FSPs_toData))
    global_tag = np.chararray((numEvents, num_FSPs_toData + 1), itemsize=30)
    
    event_list = df_num_subset[df_num_subset["numFSPs"] == num_FSPs_toData]["__event__"].unique()
    #print("len(event_list):",len(event_list))
    for i in range(numEvents):

        event_iter = event_list[i]

        global_tag_masterInfo = "evt" + str(event_iter)
        global_tag[i,-1] = global_tag_masterInfo
        #print("global_tag[i,-1]:",global_tag[i,-1])
        #print("i:",i,"event_iter:",event_iter)

        event_df = df_num_subset[df_num_subset.__event__ == event_iter]

        for j in range(num_FSPs_toData):
            #print("numParticle:",j)
            particle = event_df.iloc[j]

            #print(particle["mcPDG"],particle["px"],particle["py"],particle["pz"],particle["E"])
            leaves[i,j,0] = particle["px"]
            leaves[i,j,1] = particle["py"]
            leaves[i,j,2] = particle["pz"]
            leaves[i,j,3] = particle["E"]
            
            basf2_usage = "basf2_NONE"
            if particle["basf2_Bsig"] == 1.0:
                basf2_usage = "basf2_Bsig"
            elif particle["basf2_X"] == 1.0:
                basf2_usage = "basf2_X"
            elif particle["basf2_used"] == 0:
                basf2_usage = "basf2_bg"

            global_tag_Info = str((particle["mcPDG"])) 
            global_tag_Info += "_" + basf2_usage
            global_tag[i,j] = global_tag_Info

            label = -10 # error code if assignment fails
            B_tag_uniqID = 83886081.0
            B_sig_uniqID = 83886082.0
            if particle["B_ID"] == B_tag_uniqID:
                label = 1 # particle belongs to Btag (MC truth)
            elif particle["B_ID"] == B_sig_uniqID:
                label = 2 # particle belongs to Bsig (MC truth)
            elif particle["B_ID"] == 0:
                label = 0 # background
            
            
            SA_target[i,j] = label
            
        del event_df
        
        
    # shuffle the data    
    for idx in np.arange(leaves.shape[0]):   # arange is like range but gives ndarray instead of list
        perms = np.random.permutation(leaves.shape[1])

        leaves[idx,:] = leaves[idx,perms]
        SA_target[idx,:] = SA_target[idx,perms]
        global_tag[idx,0:-1] = global_tag[idx,perms]
        
        
         


    #print(global_tag)
    train_ratio = 0.75
    validation_ratio = 0.15
    test_ratio = 0.10

    print("leaves.shape:",leaves.shape)
    print("SA_target.shape:",SA_target.shape)
    print("global_tag.shape:",global_tag.shape)


    print("leaves[0]:",leaves[0])
    print("SA_target[0]:",SA_target[0])
    print("global_tag[0]:",global_tag[0])

    x=leaves
    y=SA_target
    z=global_tag

    x_train, x_test, y_train, y_test, z_train, z_test = train_test_split(x, y, z, test_size=1 - train_ratio, shuffle=False)
    x_val, x_test, y_val, y_test, z_val, z_test = train_test_split(x_test, y_test, z_test, test_size=test_ratio/(test_ratio + validation_ratio), shuffle=False) 

    if save_data==True:
        np.save(data_dir / "leaves_train_FSP{}.npy".format(num_FSPs_toData), x_train)
        np.save(data_dir / "is_left_arr_train_FSP{}.npy".format(num_FSPs_toData), y_train)
        np.save(data_dir / "global_tag_train_FSP{}.npy".format(num_FSPs_toData), z_train)

        np.save(data_dir / "leaves_val_FSP{}.npy".format(num_FSPs_toData), x_val)
        np.save(data_dir / "is_left_arr_val_FSP{}.npy".format(num_FSPs_toData), y_val)
        np.save(data_dir / "global_tag_val_FSP{}.npy".format(num_FSPs_toData), z_val)

        np.save(data_dir / "leaves_test_FSP{}.npy".format(num_FSPs_toData), x_test)
        np.save(data_dir / "is_left_arr_test_FSP{}.npy".format(num_FSPs_toData), y_test)
        np.save(data_dir / "global_tag_test_FSP{}.npy".format(num_FSPs_toData), z_test)

    
    print("")
    #del df_num_subset


    del df_num_subset
                                          

In [ ]:
print("saving is done")
now = datetime.now()
print("time at end =", now)